# BOOTCAMP AI GS

## Case 2

Seu time faz parte de uma start-up de grande sucesso no setor de aluguel
de casas para curtas durações. A plataforma vem experimentando um crescimento
significativo, com o site e o aplicativo atraindo cada vez mais usuários.

Atualmente, o mercado principal da empresa é a Europa, onde a oferta de
acomodações continua aumentando. Com esse crescimento acelerado, a demanda
por decisões baseadas em dados tem se intensificado, tanto para análises preditivas
quanto prescritivas.
Neste momento de expansão, foi criada uma equipe, a de Data Product
Management.

### Objetivo
 O primeiro objetivo dessa equipe é entregar um produto de dados: um
modelo de regressão capaz de prever o valor esperado do aluguel com base em
informações sobre a acomodação (como número máximo de pessoas, quantidade de
quartos, distância ao centro da cidade, latitude, longitude, dia da semana, entre outras
features).


### Análise Exploratória

Importando as bibliotecas

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

Leitura das tabelas

In [2]:
df_barcelona_weekdays =  pd.read_csv('data/barcelona_weekdays.csv')
df_barcelona_weekends=  pd.read_csv('data/barcelona_weekends.csv')

df_berlin_weekdays =  pd.read_csv('data/berlin_weekdays.csv')
df_berlin_weekends =  pd.read_csv('data/berlin_weekends.csv')

df_paris_weekdays =  pd.read_csv('data/barcelona_weekdays.csv')
df_paris_weekends =  pd.read_csv('data/barcelona_weekends.csv')

In [4]:
# Incluindo o tipo de dia

df_barcelona_weekdays["type"] = "weekdays"
df_berlin_weekdays["type"] = "weekdays"
df_paris_weekdays["type"] = "weekdays"

df_barcelona_weekends["type"] = "weekends"
df_berlin_weekends["type"] = "weekends"
df_paris_weekends["type"] = "weekends"

# Incluindo o nome da cidade

df_barcelona_weekdays["city"] = "barcelona"
df_barcelona_weekends["city"] = "barcelona"

df_berlin_weekdays["city"] = "berlin"
df_berlin_weekends["city"] = "berlin"

df_paris_weekdays["city"] = "paris"
df_paris_weekends["city"] = "paris"



Concatenando para um só dataset

In [5]:
df_original = pd.concat([df_barcelona_weekdays, df_berlin_weekdays, df_paris_weekdays, df_barcelona_weekends, df_berlin_weekends, df_paris_weekends])

In [19]:
df_original.head()

,Unnamed: 0,realSum,room_type,room_shared,room_private,person_capacity,host_is_superhost,multi,biz,cleanliness_rating,...,dist,metro_dist,attr_index,attr_index_norm,rest_index,rest_index_norm,lng,lat,type,city
0,0,474.317499,Entire home/apt,False,False,4.0,False,0,1,10.0,...,1.111996,0.630491,526.469420,17.942927,915.587083,20.154890,2.17556,41.39624,weekdays,barcelona
1,1,169.897829,Private room,False,True,2.0,True,1,0,10.0,...,1.751839,0.124017,320.127526,10.910462,794.277350,17.484489,2.14906,41.38714,weekdays,barcelona
2,2,161.984779,Private room,False,True,4.0,False,0,1,9.0,...,1.670493,0.080322,344.073936,11.726595,840.673617,18.505814,2.15357,41.37859,weekdays,barcelona
3,3,367.956804,Entire home/apt,False,False,3.0,False,0,1,10.0,...,1.475847,0.093107,400.057449,13.634603,946.589884,20.837357,2.16839,41.37390,weekdays,barcelona
4,4,196.895292,Private room,False,True,3.0,False,1,0,9.0,...,1.855452,0.272486,346.042245,11.793678,792.296039,17.440874,2.15238,41.37699,weekdays,barcelona


In [7]:
df_original.shape

(8150, 22)

Tipos de Dados

In [10]:
df_original.dtypes

Unnamed: 0                      int64
realSum                       float64
room_type                      object
room_shared                      bool
room_private                     bool
person_capacity               float64
host_is_superhost                bool
multi                           int64
biz                             int64
cleanliness_rating            float64
guest_satisfaction_overall    float64
bedrooms                        int64
dist                          float64
metro_dist                    float64
attr_index                    float64
attr_index_norm               float64
rest_index                    float64
rest_index_norm               float64
lng                           float64
lat                           float64
type                           object
city                           object
dtype: object

Valores Nulos

In [11]:
df_original.isna().sum()

Unnamed: 0                    0
realSum                       0
room_type                     0
room_shared                   0
room_private                  0
person_capacity               0
host_is_superhost             0
multi                         0
biz                           0
cleanliness_rating            0
guest_satisfaction_overall    0
bedrooms                      0
dist                          0
metro_dist                    0
attr_index                    0
attr_index_norm               0
rest_index                    0
rest_index_norm               0
lng                           0
lat                           0
type                          0
city                          0
dtype: int64

In [12]:
df_original.nunique()

Unnamed: 0                    1555
realSum                       1184
room_type                        3
room_shared                      2
room_private                     2
person_capacity                  5
host_is_superhost                2
multi                            2
biz                              2
cleanliness_rating               8
guest_satisfaction_overall      42
bedrooms                         7
dist                          5317
metro_dist                    5317
attr_index                    5317
attr_index_norm               5314
rest_index                    5317
rest_index_norm               5314
lng                           2931
lat                           2786
type                             2
city                             3
dtype: int64

In [20]:
df = df_original.copy()

continuous = []
categorical = []

for c in df.columns[:-2]:
    if df.nunique()[c]>=30:
        continuous.append(c)
    else:
        categorical.append(c)

In [21]:
continuous

['Unnamed: 0',
 'realSum',
 'guest_satisfaction_overall',
 'dist',
 'metro_dist',
 'attr_index',
 'attr_index_norm',
 'rest_index',
 'rest_index_norm',
 'lng',
 'lat']

In [22]:
categorical

['room_type',
 'room_shared',
 'room_private',
 'person_capacity',
 'host_is_superhost',
 'multi',
 'biz',
 'cleanliness_rating',
 'bedrooms']